# Baryon correlator with LapH

This notebook is a work in progress.  Important notes about limitations.

1.  The operators used are "Baryon Elemental" operators, and not necassrily definite spin (lattice irrep)/isospin.  In practice the operator construction at the start needs to be done correctly.

2.  Previously I was doing only mesons, or $N_c=4$ baryons, where the Baryon tensors "defined" below don't have any free indices.  I just need to think through what to do.  Probably most of the code in WickContractions/LapH/diagrams.py still works (and indeed T function construction does).  

3.  Need to define a "meson T function" with $\delta$ instead of $\epsilon$

In [1]:
# Avoids reloading kernel while developing
%load_ext autoreload
%autoreload 2

In [2]:
from WickContractions.wick.contract import *
from src.laph_diagram import *

import os

from src.utilities import *

#pretty latex printing
from IPython.display import display, Math 
pprint = lambda o : display(Math(str(o)))

## Operators

First we have to define the operators

In [3]:
from src.op_elementals import *

aOps=[Operator([baryonSink(1,['u','u','u'],'x_0','\\Gamma')])]
# protonOps = baryonSink([1,['u','u','d']), baryonSink([-1,['d','u','u']])
cOps=[Operator([baryonSource(1,['u','u','u'],'x_0','\\Gamma')])]

In [4]:
pprint(aOps[0])
pprint(cOps[0])


<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [6]:
diagrams=[]
for i,aOp in enumerate(aOps):
    for j,cOp in enumerate(cOps):
        for d in contract(aOp, cOp).diagrams:
            diagrams.append(d)

print("Need to compute {} diagrams".format(len(diagrams)))
pprint("\\text{{diagram}}[0]={}".format(diagrams[0]))

Need to compute 6 diagrams


<IPython.core.display.Math object>

## LapH

Here we take a short cut to replace the above expressions with their LapH versions.  LapH is of course a type of smearing applied to the quarks.  At the level of propagators we can replace inversions of the diract matrix with 

$$
    D^{-1}(x,t\mid x_0 t_0)_{\substack{s \\ c}\substack{s_0 \\ c_0}} \rightarrow V^*(x,t)_{c l}\tau(x,t\mid x_0, t_0)_{\substack{s \\ l}\substack{s_0 \\ l_0}}V(x_0,t_0)_{c_0 l_0}
$$

In [8]:
laphDiagrams = [LDiagram(d) for d in copy.deepcopy(diagrams)]

pprint(laphDiagrams[0])

<IPython.core.display.Math object>

The product of $V$'s, epsilons and gammas all come from specific operators, and be computed for the entire correlation matrix, thus it's useful to define the following tensors

$$
    T^B(x,t)_{l_0l_1l_2}=\epsilon_{c_0c_1c_2}V(x,t)_{c_0l_0}V(x,t)_{c_1l_1}V(x,t)_{c_2l_2}, \quad l_0\in N_v \\
    T^M(x,t)_{l_0l_1}=\delta_{c_0c_1}V(x,t)_{c_0l_0}V(x,t)_{c_1l_1}
$$
and
$$
    B(x,t,\Gamma)_{01 l_2}=\Gamma_{s_0s_1}T(x,t)_{l_0l_1l_2} \\
    M_{01}=\Gamma_{s_0s_1}T_{l_0l_1} \\
    0,1 \in {4\times N_v}
$$
We refer to B as a baryon block.  Usually I combine the spin and eigenvector indices into a compound index, so that that baryon tensor is a rank $N_c$ tensor with dimension $(N_c\times N_v)^3$.

For now, the code that creates $T$ functions works as expected.  Other versions look for gamma matrices with the same indices as the $T$ function, just need to think about this.

In [11]:
for d in laphDiagrams:
    d.create_T_blocks()

In [12]:
pprint(laphDiagrams[0])

<IPython.core.display.Math object>

## BPROP

Take one of the Baryon blocks and contract with perambulators

## Goal

We want to have diagrams in the following form (schematically)

$$
d^0_{lj}=B(t)_{ikl}B(0)_{kij} \\
d^1_{lj}=B(t)_{kil}B(0)_{kij} \\
$$


The computation follows the path of 

1.  Compute all the T functions (and meson ones)
2.  Compute all the M and B functions. 
3.  Contract according to the rules given by diagrams.  For the above two diagrams it would be 

* For optimization a list of diagrams can be plugged into a CSE/DC code i.e. the Ben Hortz one.
* I also write a "translator" that takes expressions like the one above and converts into a C++ code
that uses some library to evaluate the contractions.  Eigen::Tensor/cuTensor I can talk more about this/show examples.

contract( B[0], B[1], {{0,1,2},{0,1,3}} )

contract( B[0], B[1], {{1,0,2},{0,1,3}} )